In [1]:
import pandas as pd
import numpy as np
import time
import collections
import plotly.graph_objects as go
import plotly.express as px
import math

In [2]:
# Read the printify products excel file into pandas dataframe
orders = pd.read_excel('Printify_Data_Analyst_homework.xlsx', sheet_name = 'Orders')
orders.head(10)

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,2.013244e+06,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,1.358430e+06,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,9.393902e+05,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,1.856107e+06,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,1.843904e+06,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
5,6949441,1.477206e+06,1477206,US,NV,2020-10-07 05:09:46,2020-10-08 20:33:40,NaN,1,805,400,2019-12-27 22:55:01,True,Plan 4,USPS,2020-10-12 21:44:56
6,7146390,1.720307e+06,1720307,US,CO,2020-11-13 22:00:23,2020-11-15 15:52:22,NaN,1,3010,1024,2020-05-21 14:45:00,True,business_account_3,USPS,2020-11-19 20:41:30
7,6179493,6.044341e+05,604434,US,GA,2020-10-31 02:38:32,2020-11-04 20:51:54,NaN,4,1306,700,2018-10-28 05:11:39,True,Plan 4,USPS,NaT
8,7110135,1.813012e+06,1813012,US,IN,2020-11-13 04:55:57,2020-11-14 01:20:14,NaN,4,1602,624,2020-04-27 00:42:50,True,business_account_3,USPS,2020-11-19 22:49:58
9,5900409,1.325831e+06,1325831,BE,NaN,2020-11-11 11:19:05,2020-11-12 02:44:37,NaN,1,2880,1180,2018-02-23 06:30:55,True,Plan 4,NaN,NaT


In [3]:
# Read the printify products excel file into pandas dataframe
new_line_items = pd.read_excel('Printify_Data_Analyst_homework.xlsx', sheet_name = 'Line items')
new_line_items.head(30)

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,1.750015e+06,30,Generic brand,Mug,on-hold,3
1,1.750015e+06,30,Generic brand,Mug,on-hold,2
2,1.225155e+06,16,Gildan,Kids clothes,shipment_delivered,1
3,1.828891e+06,39,Bella+Canvas,T-Shirt,shipment_delivered,1
4,1.847848e+06,10,Generic brand,Accessories,shipment_delivered,1
5,1.750016e+06,30,Generic brand,Mug,on-hold,1
6,4.668482e+05,10,Generic brand,Bags,shipment_delivered,1
7,1.569960e+06,45,Gildan,Sweatshirt,fulfilled,1
8,2.004430e+06,29,Gildan,Sweatshirt,shipment_delivered,1
9,2.255664e+05,25,Bella+Canvas,T-Shirt,shipment_delivered,1


In [4]:
line_items = new_line_items.drop_duplicates(subset=['ORDER_ID', 'PRINT_PROVIDER_ID', 'PRODUCT_BRAND', 'PRODUCT_TYPE', 'ITEM_STATUS', 'QUANTITY'])
line_items.reset_index(drop=True, inplace=True)
line_items.head(30)

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,1.750015e+06,30,Generic brand,Mug,on-hold,3
1,1.750015e+06,30,Generic brand,Mug,on-hold,2
2,1.225155e+06,16,Gildan,Kids clothes,shipment_delivered,1
3,1.828891e+06,39,Bella+Canvas,T-Shirt,shipment_delivered,1
4,1.847848e+06,10,Generic brand,Accessories,shipment_delivered,1
5,1.750016e+06,30,Generic brand,Mug,on-hold,1
6,4.668482e+05,10,Generic brand,Bags,shipment_delivered,1
7,1.569960e+06,45,Gildan,Sweatshirt,fulfilled,1
8,2.004430e+06,29,Gildan,Sweatshirt,shipment_delivered,1
9,2.255664e+05,25,Bella+Canvas,T-Shirt,shipment_delivered,1


In [5]:
# for index in range(len(line_items['ORDER_ID'])):
#     whole_order = str(line_items.loc[index]['ORDER_ID'])
#     whole_order = whole_order.split(".")
#     order_id = int(whole_order[1])
#     line_items['ORDER_ID'].replace(whole_order, order_id, inplace=True)
#     # line_items.loc[index]['ORDER_ID'] = order_id
# line_items

In [6]:
# for index, order_whole in enumerate(line_items['ORDER_ID']):
#     order = str(order_whole).split(".")
#     order_id = int(order[-1])
#     print(f"{index} - {order_whole}-->{order_id}")
    
#     line_items.replace(order_whole, order_id, inplace=True)
# line_items

In [7]:
for i in range(len(line_items['ORDER_ID'])):
    order = str(line_items.loc[i]['ORDER_ID']).split(".")
    order_id = int(order[-1])
    if order_id == 0:
        print(f"{i} -- {order_id}")
    #line_items['ORDER_ID'].replace(line_items.loc[i]['ORDER_ID'], order_id, inplace=True)
    line_items.at[i, 'ORDER_ID'] = int(order[-1])
    line_items.at[i,'SHOP_ID'] = int(order[0])
line_items.head(30)

C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY,SHOP_ID
0,15208.0,30,Generic brand,Mug,on-hold,3,1750015.0
1,12357.0,30,Generic brand,Mug,on-hold,2,1750015.0
2,10081.0,16,Gildan,Kids clothes,shipment_delivered,1,1225155.0
3,707.0,39,Bella+Canvas,T-Shirt,shipment_delivered,1,1828890.0
4,55.0,10,Generic brand,Accessories,shipment_delivered,1,1847847.0
5,8111.0,30,Generic brand,Mug,on-hold,1,1750015.0
6,177.0,10,Generic brand,Bags,shipment_delivered,1,466848.0
7,13053.0,45,Gildan,Sweatshirt,fulfilled,1,1569960.0
8,117.0,29,Gildan,Sweatshirt,shipment_delivered,1,2004430.0
9,362.0,25,Bella+Canvas,T-Shirt,shipment_delivered,1,225566.0


In [8]:
line_items[line_items['ORDER_ID'] == 0]

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY,SHOP_ID


In [9]:
# for index, order_whole in enumerate(orders['ORDER_ID']):
#     order = str(order_whole).split(".")
#     # print(f"{order_whole}-->{order_id}")
#     order_id = int(order[-1])
#     orders['ORDER_ID'].replace(order_whole, order_id, inplace=True)
# orders

In [10]:
for i in range(len(orders['ORDER_ID'])):
    order = str(orders.loc[i]['ORDER_ID']).split(".")
    order_id = int(order[-1])
    if order_id == 0:
        print(f"{i} -- {order_id}")
    #line_items['ORDER_ID'].replace(line_items.loc[i]['ORDER_ID'], order_id, inplace=True)
    orders.at[i, 'ORDER_ID'] = int(order[-1])
line_items

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY,SHOP_ID
0,15208.0,30,Generic brand,Mug,on-hold,3,1750015.0
1,12357.0,30,Generic brand,Mug,on-hold,2,1750015.0
2,10081.0,16,Gildan,Kids clothes,shipment_delivered,1,1225155.0
3,707.0,39,Bella+Canvas,T-Shirt,shipment_delivered,1,1828890.0
4,55.0,10,Generic brand,Accessories,shipment_delivered,1,1847847.0
...,...,...,...,...,...,...,...
15803,1367.0,2,Generic brand,Canvas,shipment_delivered,1,1735807.0
15804,7852.0,26,Gildan,T-Shirt,fulfilled,1,1732948.0
15805,11854.0,29,Gildan,Sweatshirt,shipment_out_for_delivery,1,1449381.0
15806,2759.0,48,Generic brand,Mug,shipment_delivered,1,1735807.0


In [11]:
len(line_items['ORDER_ID'].unique())

5296

In [12]:
orders

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1,5888534,695.0,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,924,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56
2,6484224,193.0,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,400,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54
3,7251033,4.0,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,800,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37
4,7241488,4.0,1843904,US,CO,2020-11-17 19:30:10,2020-11-18 03:45:25,NaN,2,719,424,2020-07-15 18:47:51,True,business_account_3,USPS,2020-11-24 00:44:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13501,7562898,12.0,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,0,2020-10-29 16:51:47,False,NaN,NaN,NaT
13502,7475289,2224.0,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,474,2020-09-30 23:11:41,True,business_account_3,NaN,NaT
13503,7144356,5.0,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,824,2020-05-20 01:38:41,False,NaN,NaN,NaT
13504,7581194,66.0,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,824,2020-11-05 20:02:09,False,NaN,NaN,NaT


In [13]:
len(line_items['ORDER_ID'].unique())

5296

In [14]:
orders[orders['ORDER_ID'] == 0]

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT


In [15]:
# Check orders that are not in line_items dataset
dff = []
for order in line_items['ORDER_ID']:
    if order in orders['ORDER_ID']:
        dff.append(order)
len(dff)

14637

In [23]:
orders['SHOP_ID'].astype(float)

0        2013244.0
1        1358429.0
2         939390.0
3        1856107.0
4        1843904.0
           ...    
13501    2225081.0
13502    2153683.0
13503    1717607.0
13504    2243420.0
13505    2216957.0
Name: SHOP_ID, Length: 13506, dtype: float64

In [24]:
printify_df = pd.merge(orders, line_items, how="left", on=['ORDER_ID','SHOP_ID'])

In [25]:
len(printify_df['ORDER_ID'].unique())

5315

In [26]:
printify_df

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,...,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,...,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29,1.0,Generic brand,Mug,shipment_delivered,1.0
1,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,...,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29,16.0,Generic brand,Mug,shipment_delivered,1.0
2,5888534,695.0,1358429,US,MN,2020-11-21 05:19:03,2020-11-23 00:23:32,NaN,4,1149,...,2018-02-08 07:34:02,True,business_account_3,USPS,2020-11-28 21:50:56,39.0,Gildan,Sweatshirt,shipment_delivered,1.0
3,6484224,193.0,939390,US,AK,2020-10-25 01:39:08,2020-10-26 20:30:53,NaN,1,977,...,2019-04-09 20:02:46,True,business_account_3,USPS,2020-11-02 23:37:54,25.0,Delta,T-Shirt,shipment_delivered,1.0
4,7251033,4.0,1856107,US,PA,2020-10-08 16:35:21,2020-10-10 15:02:47,NaN,1,661,...,2020-07-20 17:04:41,True,business_account_3,UPS,2020-10-15 18:08:37,16.0,Generic brand,Mug,shipment_delivered,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15867,7562898,12.0,2225081,CA,ON,2020-11-05 21:12:29,NaT,NaN,1,0,...,2020-10-29 16:51:47,False,NaN,NaN,NaT,10.0,Generic brand,Accessories,canceled,10.0
15868,7475289,2224.0,2153683,US,CO,2020-11-16 04:11:43,NaT,NaN,1,1447,...,2020-09-30 23:11:41,True,business_account_3,NaN,NaT,16.0,Sport-Tek,Hats,on-hold,1.0
15869,7144356,5.0,1717607,US,CT,2020-12-25 00:08:07,NaT,NaN,1,2187,...,2020-05-20 01:38:41,False,NaN,NaN,NaT,3.0,Gildan,Hoodie,on-hold,1.0
15870,7581194,66.0,2243420,US,IN,2020-11-19 16:43:04,NaT,NaN,4,2187,...,2020-11-05 20:02:09,False,NaN,NaN,NaT,3.0,Gildan,Hoodie,canceled,1.0


In [27]:
orders[orders['ORDER_ID'] == 178.0]

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,TOTAL_SHIPPING,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT
0,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,2200,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29
1233,6995143,178.0,1528802,US,CO,2020-11-12 23:12:27,2020-11-13 14:18:45,NaN,2,1503,524,2020-02-02 05:29:14,True,Plan 4,USPS,2020-11-17 03:02:32
3927,5866778,178.0,230009,US,WA,2020-11-27 16:41:59,2020-11-30 06:43:19,NaN,4,4177,2524,2018-01-14 23:58:54,False,NaN,DHL,2020-12-10 00:36:47
4013,7550434,178.0,2206703,US,SC,2020-12-10 15:23:57,2020-12-15 03:06:25,NaN,4,661,424,2020-10-25 14:21:29,False,NaN,USPS,2020-12-18 20:50:54
7973,7367531,178.0,1995997,CA,BC,2020-11-22 22:09:41,2020-11-24 21:52:46,NaN,4,1409,615,2020-09-03 08:37:44,False,NaN,CANADA_POST,2020-11-30 23:57:15
8760,7113319,178.0,1677803,US,MA,2020-11-28 01:03:53,2020-12-03 15:35:53,NaN,4,2125,924,2020-04-29 01:58:44,False,NaN,USPS,2020-12-03 17:27:52
9149,6548805,178.0,1009173,US,CA,2020-10-26 02:44:33,2020-10-27 15:19:49,NaN,2,757,400,2019-05-01 08:23:09,True,business_account_3,USPS,2020-10-30 22:42:26
9731,7220332,178.0,1818877,US,CA,2020-12-14 19:45:50,2020-12-17 10:43:26,NaN,4,1229,524,2020-07-04 23:26:44,False,NaN,USPS,2020-12-19 23:08:56
11841,7266148,178.0,1874973,US,MI,2020-10-10 18:22:34,NaT,NaN,4,0,0,2020-07-27 19:20:37,False,NaN,NaN,NaT
12612,7475289,178.0,2153683,US,NC,2020-10-17 17:16:38,2020-10-23 23:10:16,NaN,1,2713,940,2020-09-30 23:11:41,True,business_account_3,USPS,2020-10-30 22:39:30


In [28]:
printify_df[printify_df['ORDER_ID'] == 178.0]

,MERCHANT_ID,ORDER_ID,SHOP_ID,ADDRESS_TO_COUNTRY,ADDRESS_TO_REGION,ORDER_DT,FULFILLED_DT,REPRINT_FLAG,SALES_CHANNEL_TYPE_ID,TOTAL_COST,...,MERCHANT_REGISTERED_DT,SUB_IS_ACTIVE_FLAG,SUB_PLAN,SHIPMENT_CARRIER,SHIPMENT_DELIVERD_DT,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY
0,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,...,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29,1.0,Generic brand,Mug,shipment_delivered,1.0
1,7143578,178.0,2013244,US,WA,2020-10-02 19:30:15,2020-10-20 07:53:06,NaN,9,2902,...,2020-05-19 15:22:57,True,business_account_3,UPS,2020-10-08 21:07:29,16.0,Generic brand,Mug,shipment_delivered,1.0
1459,6995143,178.0,1528802,US,CO,2020-11-12 23:12:27,2020-11-13 14:18:45,NaN,2,1503,...,2020-02-02 05:29:14,True,Plan 4,USPS,2020-11-17 03:02:32,10.0,Generic brand,Accessories,shipment_delivered,1.0
4622,5866778,178.0,230009,US,WA,2020-11-27 16:41:59,2020-11-30 06:43:19,NaN,4,4177,...,2018-01-14 23:58:54,False,NaN,DHL,2020-12-10 00:36:47,30.0,Generic brand,Paper products,fulfilled,1.0
4623,5866778,178.0,230009,US,WA,2020-11-27 16:41:59,2020-11-30 06:43:19,NaN,4,4177,...,2018-01-14 23:58:54,False,NaN,DHL,2020-12-10 00:36:47,16.0,Sport-Tek,T-Shirt,shipment_delivered,1.0
4624,5866778,178.0,230009,US,WA,2020-11-27 16:41:59,2020-11-30 06:43:19,NaN,4,4177,...,2018-01-14 23:58:54,False,NaN,DHL,2020-12-10 00:36:47,1.0,Generic brand,All Over Prints,shipment_delivered,1.0
4728,7550434,178.0,2206703,US,SC,2020-12-10 15:23:57,2020-12-15 03:06:25,NaN,4,661,...,2020-10-25 14:21:29,False,NaN,USPS,2020-12-18 20:50:54,29.0,Gildan,T-Shirt,shipment_out_for_delivery,1.0
9339,7367531,178.0,1995997,CA,BC,2020-11-22 22:09:41,2020-11-24 21:52:46,NaN,4,1409,...,2020-09-03 08:37:44,False,NaN,CANADA_POST,2020-11-30 23:57:15,41.0,Bella+Canvas,T-Shirt,shipment_delivered,1.0
10268,7113319,178.0,1677803,US,MA,2020-11-28 01:03:53,2020-12-03 15:35:53,NaN,4,2125,...,2020-04-29 01:58:44,False,NaN,USPS,2020-12-03 17:27:52,45.0,Rabbit Skins,Kids clothes,shipment_delivered,1.0
10269,7113319,178.0,1677803,US,MA,2020-11-28 01:03:53,2020-12-03 15:35:53,NaN,4,2125,...,2020-04-29 01:58:44,False,NaN,USPS,2020-12-03 17:27:52,16.0,Rabbit Skins,Kids clothes,shipment_delivered,1.0


In [29]:
line_items[line_items['ORDER_ID'] == 178.0]

,ORDER_ID,PRINT_PROVIDER_ID,PRODUCT_BRAND,PRODUCT_TYPE,ITEM_STATUS,QUANTITY,SHOP_ID
409,178.0,52,Gildan,T-Shirt,canceled,1,1874973.0
2963,178.0,1,Generic brand,Mug,shipment_delivered,1,2013244.0
2964,178.0,16,Generic brand,Mug,shipment_delivered,1,2013244.0
7480,178.0,3,Bella+Canvas,T-Shirt,shipment_delivered,1,1009173.0
7668,178.0,30,Generic brand,Paper products,fulfilled,1,230009.0
7669,178.0,16,Sport-Tek,T-Shirt,shipment_delivered,1,230009.0
7670,178.0,1,Generic brand,All Over Prints,shipment_delivered,1,230009.0
8085,178.0,10,Generic brand,Accessories,shipment_delivered,1,1528802.0
8959,178.0,45,Rabbit Skins,Kids clothes,shipment_delivered,1,1677803.0
8960,178.0,16,Rabbit Skins,Kids clothes,shipment_delivered,1,1677803.0


In [30]:
len(printify_df[['ORDER_ID','PRODUCT_BRAND','PRODUCT_TYPE']].duplicated(keep='first'))

15872